In [8]:
%%writefile Airbnb.py

import pymongo
from pymongo import MongoClient
import pandas as pd
import bson.decimal128 as Decimal128
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import streamlit as st

# Connect to MongoDBajori
client = MongoClient("mongodb+srv://<username>:<password>@cluster0.4adr18k.mongodb.net/?retryWrites=true&w=majority")
db = client['sample_airbnb']
# Store channel details in collection
collection = db["listingsAndReviews"]

# Assuming you have established a connection to MongoDB Atlas and have the collection

cursor = collection.find()
df = pd.DataFrame(list(cursor))

from bson.decimal128 import Decimal128

def preprocess_dataframe(df):
    """
    Preprocess the dataframe:
    1. Convert Decimal128 columns to float.
    2. Convert list columns to string.
    3. Convert dictionary columns to string.
    4. Drop duplicates.
    """
    
    # Convert Decimal128 columns to float
    decimal128_columns = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, Decimal128)).any()]
    for col in decimal128_columns:
        df[col] = df[col].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)

    # Convert list columns to string
    list_columns = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, list)).any()]
    for col in list_columns:
        df[col] = df[col].apply(lambda x: str(x) if isinstance(x, list) else x)

    # Convert dictionary columns to string and drop duplicates
    print("Number of rows before removing duplicates:", len(df))
    
    dict_columns = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, dict)).any()]
    for col in dict_columns:
        df[col] = df[col].apply(lambda x: str(x) if isinstance(x, dict) else x)

    df.drop_duplicates(inplace=True)

    # Convert the string representation back to dictionaries
    for col in dict_columns:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("{") else x)

    print("Number of rows after removing duplicates:", len(df))
    
    return df


#preprocessed_df = preprocess_dataframe(df)

def clean_dataframe(df):
    """
    Clean the dataframe:
    1. Convert amenities lists to strings.
    2. Create a new column with the number of reviews.
    3. Fill missing values in numeric columns with median.
    4. Fill missing values in non-numeric columns with mode (or empty list for 'reviews' column).
    """
    
    # Convert amenities lists to strings
    df['amenities'] = df['amenities'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

    # Create a new column with the number of reviews
    df['number_of_reviews'] = df['reviews'].apply(lambda x: len(x) if isinstance(x, list) else 0)

    # Check for missing values before cleaning
    print("Missing values before cleaning:\n", df.isnull().sum())

    # Separate columns into numeric and non-numeric
    numeric_cols = df.select_dtypes(include=['number']).columns
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns

    # Fill missing values in numeric columns with median
    for col in numeric_cols:
        df[col].fillna(df[col].median(), inplace=True)

    for col in non_numeric_cols:
        if col == 'reviews':
            df['reviews'] = df['reviews'].apply(lambda x: [] if isinstance(x, (float, int)) and pd.isna(x) else x)
        else:
            mode_series = df[col].mode()
            mode_value = mode_series.iloc[0] if not mode_series.empty else 'Unknown'
            df[col].fillna(mode_value, inplace=True)

    # Check for any remaining null values after cleaning
    print("\nMissing values after cleaning:\n", df.isnull().sum())
    
    return df


#cleaned_df = clean_dataframe(df)

# Extract latitude and longitude from the nested dictionaries
df['latitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[1] if isinstance(x, dict) else None)
df['longitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[0] if isinstance(x, dict) else None)

# Checking the first few rows of the extracted latitude and longitude columns
df[['latitude', 'longitude']].head()


def plot_map_with_price(df):
    """
    1. Clean the 'price' column by converting it to a numeric column and removing non-numeric characters.
    2. Fill missing values in the 'price' column with the median.
    3. Plot a scatter map with the 'price' as the color indicator.
    """
    
    # Convert Decimal128 to float
    df['price'] = df['price'].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)
    
    # Convert the 'price' column to numeric, removing any non-numeric characters
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

    # Handle missing values
    df['price'].fillna(df['price'].median(), inplace=True)

    # Plotting the map
    fig = px.scatter_mapbox(df,
                            lat='latitude',
                            lon='longitude',
                            color='price',
                            color_continuous_scale=px.colors.sequential.Plasma,
                            hover_data=['name', 'price'],
                            size_max=15,
                            zoom=10,
                            mapbox_style="carto-positron")

    # Set the mapbox style
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

    return fig

#fig = plot_map_with_price(df)
def plot_avg_price_by_property_type(df):
    """
    1. Convert the 'last_review' column to a datetime format.
    2. Calculate the average price by property type.
    3. Plot an interactive bar chart of average price by property type.
    """
    
    # Convert 'last_review' to datetime
    df['last_review'] = pd.to_datetime(df['last_review'])
    
    # Convert Decimal128 to float for the 'price' column
    df['price'] = df['price'].apply(lambda x: float(x.to_decimal()) if isinstance(x, Decimal128) else x)

    # Calculate average price by property type for plotting
    avg_prices = df.groupby('property_type')['price'].mean().reset_index()

    # Create an interactive bar plot using Plotly
    fig = px.bar(avg_prices, 
                 x='property_type', 
                 y='price', 
                 color='price',
                 labels={'price': 'Average Price', 'property_type': 'Property Type'},
                 title='Price Analysis by Property Type', 
                 hover_data=['property_type', 'price'],
                 color_continuous_scale='Plasma')

    return fig
#plot_avg_price_by_property_type(df)

def plot_avg_price_by_month(df):
    """
    1. Extract the month from the 'last_review' column.
    2. Calculate the average price by month.
    3. Plot an interactive pie chart of average price by month.
    """
    
    # Extract month from 'last_review' and map it to month names
    df['month'] = df['last_review'].dt.month_name()

    # Calculate average price by month for plotting
    avg_prices_per_month = df.groupby('month')['price'].mean().reset_index()

    # Create an interactive pie chart using Plotly
    fig = px.pie(avg_prices_per_month, names='month', values='price', title='Average Price Analysis by Month')

    #fig.show()

    return fig


#fig = plot_avg_price_by_month(df)

def plot_avg_price_by_suburb(df):
    """
    1. Extract the suburb from the 'address' column.
    2. Calculate the average price by suburb.
    3. Plot an interactive bar chart of average price by suburb.
    """
    
    # Extract suburb from the nested address dictionary
    df['suburb'] = df['address'].apply(lambda x: x.get('suburb') if isinstance(x, dict) else None)

    # Calculate average price by suburb for plotting
    avg_prices_suburb = df.groupby('suburb')['price'].mean().reset_index()

    # Create an interactive bar plot using Plotly
    fig = px.bar(avg_prices_suburb, x='suburb', y='price',
                 color='price',
                 labels={'price': 'Average Price', 'suburb': 'Suburb'},
                 title='Average Price Analysis by Suburb',
                 hover_data=['suburb', 'price'],
                 color_continuous_scale='Plasma')
    fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

    #fig.show()

    return fig


#fig = plot_avg_price_by_suburb(df)

def plot_correlation_heatmap(df, exclude_columns=[], vmin=-1, vmax=1):
    """
    Plot a correlation heatmap.
    
    Parameters:
    - df: DataFrame with the data
    - exclude_columns: List of columns to exclude from the heatmap
    - vmin: Minimum value for the color scale
    - vmax: Maximum value for the color scale
    
    Returns:
    - None
    """
    # Exclude specified columns
    numeric_df = df.select_dtypes(include=[np.number]).drop(columns=exclude_columns)

    # Compute the correlation matrix
    corr_matrix = numeric_df.corr()

    # Visualize the correlation matrix using a heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, vmin=vmin, vmax=vmax)
    plt.title('Price Correlation Heatmap')
    plt.tight_layout()
    plt.show()



# Usage:
#plot_correlation_heatmap(df, exclude_columns=['number_of_reviews'])





def plot_boxplots_for_outliers(df, columns_to_check=['price', 'weekly_price', 'monthly_price']):
    """
    Plot boxplots for specified columns to visually check for outliers.
    
    Parameters:
    - df: DataFrame containing the data
    - columns_to_check: List of columns to plot boxplots for
    
    Returns:
    - None
    """
    plt.figure(figsize=(15, 7))

    for i, col in enumerate(columns_to_check, 1):
        plt.subplot(1, len(columns_to_check), i)  # Adjust the subplot layout based on the number of columns
        sns.boxplot(y=df[col])
        plt.title(f'Box plot for {col}')
        plt.ylabel(col)

    plt.tight_layout()
    plt.show()



# Usage:
#plot_boxplots_for_outliers(df)




def plot_seasonal_availability(df):
    """
    Extract availability data and plot seasonal availability of Airbnb listings.
    
    Parameters:
    - df: DataFrame containing the data with an 'availability' column
    
    Returns:
    - None
    """
    # Extracting availability data for 30, 60, 90, and 365 days
    df['availability_30'] = df['availability'].apply(lambda x: x['availability_30'])
    df['availability_60'] = df['availability'].apply(lambda x: x['availability_60'])
    df['availability_90'] = df['availability'].apply(lambda x: x['availability_90'])
    df['availability_365'] = df['availability'].apply(lambda x: x['availability_365'])

    # Define seasons based on Northern Hemisphere
    seasons = {
        "Winter": [1, 2],
        "Spring": [3, 4, 5],
        "Summer": [6, 7, 8],
        "Fall": [9, 10, 11],
        "Annual": [12]
    }

    # Calculate average availability for each season
    seasonal_availability = {}
    for season, months in seasons.items():
        if len(months) == 2:
            seasonal_availability[season] = df['availability_60'].mean()
        elif len(months) == 3:
            seasonal_availability[season] = df['availability_90'].mean()
        elif len(months) == 1 and 12 in months:
            seasonal_availability[season] = df['availability_365'].mean()

    # Visualize the seasonal availability
    plt.bar(seasonal_availability.keys(), seasonal_availability.values(), color=['blue', 'green', 'yellow', 'orange', 'red'])
    plt.ylabel('Average Availability')
    plt.title('Seasonal Availability of Airbnb Listings')
    plt.tight_layout()
    plt.show()



# Usage:
#plot_seasonal_availability(df)



def generate_map_by_neighborhood_avg_price(collection):
    """
    Generate a scatter map based on average price by listing location for each neighborhood.
    
    Parameters:
    - collection: MongoDB collection containing the listings data
    
    Returns:
    - fig: Plotly scatter map
    """
    pipeline = [
        {
            "$group": {
                "_id": "$address.suburb", 
                "average_price": {"$avg": "$price"},
                "coordinates": {"$push": "$address.location.coordinates"}
            }
        },
        {
            "$sort": {"average_price": -1}  # Sorting in descending order based on average price
        }
    ]

    results = list(collection.aggregate(pipeline))
    
    # Expand results to a flat DataFrame
    data = []
    for item in results:
        neighborhood = item['_id']
        avg_price = item['average_price']
        for coord in item['coordinates']:
            data.append({
                "neighborhood": neighborhood,
                "average_price": avg_price,
                "longitude": coord[0],
                "latitude": coord[1]
            })

    df_results = pd.DataFrame(data)

    # Convert the average_price column from Decimal128 to string, then to float
    df_results['average_price'] = df_results['average_price'].apply(lambda x: float(str(x)))

    # Generate the scatter map
    fig = px.scatter_mapbox(df_results, 
                            lat="latitude", 
                            lon="longitude", 
                            color="average_price",
                            hover_name="neighborhood",
                            hover_data=["average_price"],
                            color_continuous_scale=px.colors.cyclical.IceFire, 
                            title="Average Price by Listing Location",
                            mapbox_style="carto-positron",
                            zoom=10,
                            size="average_price",
                            size_max=15)

    return fig



# Usage:
#fig = generate_map_by_neighborhood_avg_price(collection)
#fig.show()



# ... [Your previous imports and data retrieval/preprocessing functions]

def main():
    st.title("Airbnb Data Analysis Dashboard")
    
    df['suburb'] = df['address'].apply(lambda x: x.get('suburb') if isinstance(x, dict) else None)

    
    menu = ["Home", "Geospatial Visualization", "Price Analysis and Visualization", "Availability Analysis by Season", "Location-Based Insights"]
    choice = st.sidebar.selectbox("Menu", menu)
    
    # Sidebar with filtering options
    st.sidebar.header("Filter Options")
    selected_property_type = st.sidebar.selectbox("Choose a Property Type", df['property_type'].unique())
    selected_neighborhood = st.sidebar.selectbox("Choose a Neighborhood", df['suburb'].unique())
    selected_date_range = st.sidebar.date_input("Pick a date range", [df['last_review'].min(), df['last_review'].max()])

    # Convert the Python date objects to pandas Timestamps
    start_date = pd.Timestamp(selected_date_range[0])
    end_date = pd.Timestamp(selected_date_range[1])

    # Filtering data based on user's selection
    filtered_data = df[
        (df['property_type'] == selected_property_type) & 
        (df['suburb'] == selected_neighborhood) & 
        (df['last_review'] >= start_date) & 
        (df['last_review'] <= end_date)
    ]
    
    if choice == "Home":
        st.write("Welcome to the Airbnb Data Analysis Dashboard!")
        st.write("Please select a category from the sidebar to view visualizations and insights.")
        
    elif choice == "Geospatial Visualization":
        st.subheader("Geospatial Visualization")
        fig_geo = plot_map_with_price(filtered_data)
        st.plotly_chart(fig_geo)
        
    elif choice == "Price Analysis and Visualization":
        st.subheader("Price Analysis by Property Type")
        fig_property = plot_avg_price_by_property_type(df)
        fig_property1 = plot_avg_price_by_property_type(filtered_data)
        st.plotly_chart(fig_property)
        st.plotly_chart(fig_property1)


        st.subheader("Average Price Analysis by Month")
        fig_month = plot_avg_price_by_month(df)
        fig_month1 = plot_avg_price_by_month(filtered_data)
        st.plotly_chart(fig_month)
        st.plotly_chart(fig_month1)

        st.subheader("Average Price Analysis by Suburb")
        fig_suburb = plot_avg_price_by_suburb(df)
        fig_suburb1 = plot_avg_price_by_suburb(filtered_data)
        st.plotly_chart(fig_suburb)
        st.plotly_chart(fig_suburb1)
        
    elif choice == "Availability Analysis by Season":
        st.set_option('deprecation.showPyplotGlobalUse', False)
        st.subheader("Availability Analysis by Season")
        fig_season = plot_seasonal_availability(df)
        fig_season1 = plot_seasonal_availability(filtered_data)
        st.pyplot(fig_season)
        st.pyplot(fig_season1)
        
        
    elif choice == "Location-Based Insights":
        st.set_option('deprecation.showPyplotGlobalUse', False)
        st.subheader("Average Price by Listing Location")
        fig_location = generate_map_by_neighborhood_avg_price(collection)
        st.plotly_chart(fig_location)
        

if __name__ == "__main__":
    main()


Overwriting Airbnb.py


In [9]:
%run airbnb.py